## Part 1: CUDA C++ (Primary)

### GPU Check

In [ ]:
!nvcc --version
!nvidia-smi --query-gpu=name,memory.total --format=csv

### Memory Hierarchy Review

```
┌─────────────────────────────────────────────────────────────────┐
│                     GPU Memory Hierarchy                        │
├─────────────────────────────────────────────────────────────────┤
│  Registers        Per-thread, fastest, ~255 per thread         │
├─────────────────────────────────────────────────────────────────┤
│  Shared Memory    Per-block, ~48-164KB, programmer-managed     │
├─────────────────────────────────────────────────────────────────┤
│  Constant Memory  64KB, cached, broadcast to warp    ← TODAY   │
├─────────────────────────────────────────────────────────────────┤
│  Texture Memory   Cached, 2D spatial locality        ← TODAY   │
├─────────────────────────────────────────────────────────────────┤
│  L1/L2 Cache      Automatic caching                            │
├─────────────────────────────────────────────────────────────────┤
│  Global Memory    Main VRAM, 4-80GB, slowest                   │
└─────────────────────────────────────────────────────────────────┘
```

### Constant Memory

**Constant memory** is:
- 64KB total (device-wide)
- Cached in a special constant cache
- **Broadcast to all threads in a warp** when they read the same address
- Read-only from device code

**Best for:** Small data that ALL threads read (like convolution kernels, lookup tables)

In [ ]:
%%writefile constant_memory.cu
/**
 * Constant Memory Demo
 * 
 * Shows how to use constant memory for broadcast data.
 */

#include <stdio.h>
#include <cuda_runtime.h>

#define N (1 << 22)  // 4M elements
#define FILTER_SIZE 9
#define ITERATIONS 100

// Constant memory declaration (at file scope)
__constant__ float d_filter[FILTER_SIZE];

// Using constant memory
__global__ void applyFilterConstant(float *input, float *output, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        float sum = 0.0f;
        for (int i = 0; i < FILTER_SIZE; i++) {
            // All threads read same d_filter[i] → broadcast!
            int input_idx = (idx + i) % n;
            sum += input[input_idx] * d_filter[i];
        }
        output[idx] = sum;
    }
}

// Using global memory (for comparison)
__global__ void applyFilterGlobal(float *input, float *output, float *filter, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        float sum = 0.0f;
        for (int i = 0; i < FILTER_SIZE; i++) {
            int input_idx = (idx + i) % n;
            sum += input[input_idx] * filter[i];  // Global memory access
        }
        output[idx] = sum;
    }
}

int main() {
    printf("=== Constant Memory Demo ===\n");
    printf("Array size: %d, Filter size: %d\n\n", N, FILTER_SIZE);
    
    // Host filter
    float h_filter[FILTER_SIZE];
    for (int i = 0; i < FILTER_SIZE; i++) {
        h_filter[i] = 1.0f / FILTER_SIZE;  // Simple averaging filter
    }
    
    // Copy to constant memory
    cudaMemcpyToSymbol(d_filter, h_filter, FILTER_SIZE * sizeof(float));
    
    // Device memory
    float *d_input, *d_output, *d_filter_global;
    cudaMalloc(&d_input, N * sizeof(float));
    cudaMalloc(&d_output, N * sizeof(float));
    cudaMalloc(&d_filter_global, FILTER_SIZE * sizeof(float));
    
    // Initialize
    float *h_input = (float*)malloc(N * sizeof(float));
    for (int i = 0; i < N; i++) h_input[i] = 1.0f;
    cudaMemcpy(d_input, h_input, N * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_filter_global, h_filter, FILTER_SIZE * sizeof(float), cudaMemcpyHostToDevice);
    
    int threads = 256;
    int blocks = (N + threads - 1) / threads;
    
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    
    // Benchmark constant memory
    applyFilterConstant<<<blocks, threads>>>(d_input, d_output, N);
    cudaDeviceSynchronize();
    
    cudaEventRecord(start);
    for (int i = 0; i < ITERATIONS; i++) {
        applyFilterConstant<<<blocks, threads>>>(d_input, d_output, N);
    }
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    
    float constant_time;
    cudaEventElapsedTime(&constant_time, start, stop);
    constant_time /= ITERATIONS;
    
    // Benchmark global memory
    applyFilterGlobal<<<blocks, threads>>>(d_input, d_output, d_filter_global, N);
    cudaDeviceSynchronize();
    
    cudaEventRecord(start);
    for (int i = 0; i < ITERATIONS; i++) {
        applyFilterGlobal<<<blocks, threads>>>(d_input, d_output, d_filter_global, N);
    }
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    
    float global_time;
    cudaEventElapsedTime(&global_time, start, stop);
    global_time /= ITERATIONS;
    
    printf("✅ Constant Memory: %.3f ms\n", constant_time);
    printf("   Global Memory:   %.3f ms\n", global_time);
    printf("\nSpeedup: %.2fx\n", global_time / constant_time);
    
    printf("\n💡 Constant memory wins when all threads read the same data!\n");
    
    cudaFree(d_input);
    cudaFree(d_output);
    cudaFree(d_filter_global);
    free(h_input);
    
    return 0;
}

In [ ]:
!nvcc -arch=sm_75 -O3 constant_memory.cu -o constant_memory && ./constant_memory

### Constant Memory API

```cpp
// Declare at file scope (outside functions)
__constant__ float d_data[SIZE];

// Copy from host to constant memory
cudaMemcpyToSymbol(d_data, h_data, size);

// Copy from constant memory to host
cudaMemcpyFromSymbol(h_data, d_data, size);
```

### Texture Memory (and Modern Alternatives)

**Texture memory** was traditionally used for:
- 2D/3D spatial locality caching
- Hardware interpolation
- Boundary handling (clamp, wrap)

**Modern approach:** Use `__ldg()` (load via texture cache) for simple read-only data.

```cpp
// Old way: complex texture object setup
// New way: simple __ldg() intrinsic
float val = __ldg(&input[idx]);  // Uses texture cache automatically
```

In [ ]:
%%writefile ldg_cache.cu
/**
 * Using __ldg() for read-only data
 * 
 * The __ldg() intrinsic loads through the texture cache,
 * which can improve performance for read-only data.
 */

#include <stdio.h>
#include <cuda_runtime.h>

#define N (1 << 24)
#define ITERATIONS 100

// Normal global memory load
__global__ void normalLoad(const float* __restrict__ input, float *output, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        output[idx] = input[idx] * 2.0f;
    }
}

// Load through texture cache via __ldg()
__global__ void ldgLoad(const float* __restrict__ input, float *output, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        output[idx] = __ldg(&input[idx]) * 2.0f;
    }
}

int main() {
    printf("=== __ldg() Texture Cache Demo ===\n");
    printf("Array size: %d elements\n\n", N);
    
    float *d_input, *d_output;
    cudaMalloc(&d_input, N * sizeof(float));
    cudaMalloc(&d_output, N * sizeof(float));
    
    int threads = 256;
    int blocks = (N + threads - 1) / threads;
    
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    
    // Benchmark normal
    normalLoad<<<blocks, threads>>>(d_input, d_output, N);
    cudaDeviceSynchronize();
    
    cudaEventRecord(start);
    for (int i = 0; i < ITERATIONS; i++) {
        normalLoad<<<blocks, threads>>>(d_input, d_output, N);
    }
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    
    float normal_time;
    cudaEventElapsedTime(&normal_time, start, stop);
    normal_time /= ITERATIONS;
    
    // Benchmark __ldg
    ldgLoad<<<blocks, threads>>>(d_input, d_output, N);
    cudaDeviceSynchronize();
    
    cudaEventRecord(start);
    for (int i = 0; i < ITERATIONS; i++) {
        ldgLoad<<<blocks, threads>>>(d_input, d_output, N);
    }
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    
    float ldg_time;
    cudaEventElapsedTime(&ldg_time, start, stop);
    ldg_time /= ITERATIONS;
    
    printf("Normal load:  %.3f ms\n", normal_time);
    printf("__ldg() load: %.3f ms\n", ldg_time);
    printf("\nNote: Modern compilers often auto-optimize read-only loads.\n");
    printf("The __restrict__ keyword helps the compiler use __ldg() automatically.\n");
    
    cudaFree(d_input);
    cudaFree(d_output);
    
    return 0;
}

In [ ]:
!nvcc -arch=sm_75 -O3 ldg_cache.cu -o ldg_cache && ./ldg_cache

### 2D Image Convolution Example

Putting it all together: constant memory for filter + shared memory for tiling.

In [ ]:
%%writefile convolution_2d.cu
/**
 * 2D Convolution with Constant Memory (Filter) + Shared Memory (Tile)
 * 
 * A practical example combining Week 2 concepts.
 */

#include <stdio.h>
#include <cuda_runtime.h>

#define WIDTH 1024
#define HEIGHT 1024
#define FILTER_RADIUS 1
#define FILTER_SIZE (2 * FILTER_RADIUS + 1)
#define TILE_SIZE 16
#define BLOCK_SIZE (TILE_SIZE + 2 * FILTER_RADIUS)

// Filter in constant memory
__constant__ float d_filter[FILTER_SIZE * FILTER_SIZE];

// Naive convolution (no optimizations)
__global__ void convolutionNaive(float *output, float *input, int width, int height) {
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    
    if (col < width && row < height) {
        float sum = 0.0f;
        for (int fy = -FILTER_RADIUS; fy <= FILTER_RADIUS; fy++) {
            for (int fx = -FILTER_RADIUS; fx <= FILTER_RADIUS; fx++) {
                int imgRow = min(max(row + fy, 0), height - 1);
                int imgCol = min(max(col + fx, 0), width - 1);
                int filterIdx = (fy + FILTER_RADIUS) * FILTER_SIZE + (fx + FILTER_RADIUS);
                sum += input[imgRow * width + imgCol] * d_filter[filterIdx];
            }
        }
        output[row * width + col] = sum;
    }
}

// Optimized: shared memory tiling
__global__ void convolutionTiled(float *output, float *input, int width, int height) {
    __shared__ float tile[BLOCK_SIZE][BLOCK_SIZE];
    
    int tx = threadIdx.x;
    int ty = threadIdx.y;
    
    int col = blockIdx.x * TILE_SIZE + tx - FILTER_RADIUS;
    int row = blockIdx.y * TILE_SIZE + ty - FILTER_RADIUS;
    
    // Load tile with halo
    int clampedRow = min(max(row, 0), height - 1);
    int clampedCol = min(max(col, 0), width - 1);
    tile[ty][tx] = input[clampedRow * width + clampedCol];
    
    __syncthreads();
    
    // Only interior threads compute output
    if (tx >= FILTER_RADIUS && tx < BLOCK_SIZE - FILTER_RADIUS &&
        ty >= FILTER_RADIUS && ty < BLOCK_SIZE - FILTER_RADIUS) {
        
        int outCol = blockIdx.x * TILE_SIZE + (tx - FILTER_RADIUS);
        int outRow = blockIdx.y * TILE_SIZE + (ty - FILTER_RADIUS);
        
        if (outCol < width && outRow < height) {
            float sum = 0.0f;
            for (int fy = -FILTER_RADIUS; fy <= FILTER_RADIUS; fy++) {
                for (int fx = -FILTER_RADIUS; fx <= FILTER_RADIUS; fx++) {
                    int filterIdx = (fy + FILTER_RADIUS) * FILTER_SIZE + (fx + FILTER_RADIUS);
                    sum += tile[ty + fy][tx + fx] * d_filter[filterIdx];
                }
            }
            output[outRow * width + outCol] = sum;
        }
    }
}

int main() {
    printf("=== 2D Convolution: Naive vs Tiled ===\n");
    printf("Image: %dx%d, Filter: %dx%d\n\n", WIDTH, HEIGHT, FILTER_SIZE, FILTER_SIZE);
    
    // Gaussian blur filter
    float h_filter[FILTER_SIZE * FILTER_SIZE] = {
        1/16.0f, 2/16.0f, 1/16.0f,
        2/16.0f, 4/16.0f, 2/16.0f,
        1/16.0f, 2/16.0f, 1/16.0f
    };
    cudaMemcpyToSymbol(d_filter, h_filter, FILTER_SIZE * FILTER_SIZE * sizeof(float));
    
    size_t size = WIDTH * HEIGHT * sizeof(float);
    float *d_input, *d_output;
    cudaMalloc(&d_input, size);
    cudaMalloc(&d_output, size);
    
    // Initialize with test pattern
    float *h_input = (float*)malloc(size);
    for (int i = 0; i < WIDTH * HEIGHT; i++) h_input[i] = (i % 256) / 255.0f;
    cudaMemcpy(d_input, h_input, size, cudaMemcpyHostToDevice);
    
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    
    // Benchmark naive
    dim3 blockNaive(16, 16);
    dim3 gridNaive((WIDTH + 15) / 16, (HEIGHT + 15) / 16);
    
    convolutionNaive<<<gridNaive, blockNaive>>>(d_output, d_input, WIDTH, HEIGHT);
    cudaDeviceSynchronize();
    
    cudaEventRecord(start);
    for (int i = 0; i < 100; i++) {
        convolutionNaive<<<gridNaive, blockNaive>>>(d_output, d_input, WIDTH, HEIGHT);
    }
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    
    float naive_time;
    cudaEventElapsedTime(&naive_time, start, stop);
    naive_time /= 100;
    
    // Benchmark tiled
    dim3 blockTiled(BLOCK_SIZE, BLOCK_SIZE);
    dim3 gridTiled((WIDTH + TILE_SIZE - 1) / TILE_SIZE, (HEIGHT + TILE_SIZE - 1) / TILE_SIZE);
    
    convolutionTiled<<<gridTiled, blockTiled>>>(d_output, d_input, WIDTH, HEIGHT);
    cudaDeviceSynchronize();
    
    cudaEventRecord(start);
    for (int i = 0; i < 100; i++) {
        convolutionTiled<<<gridTiled, blockTiled>>>(d_output, d_input, WIDTH, HEIGHT);
    }
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    
    float tiled_time;
    cudaEventElapsedTime(&tiled_time, start, stop);
    tiled_time /= 100;
    
    printf("❌ Naive (constant mem only): %.3f ms\n", naive_time);
    printf("✅ Tiled (constant + shared): %.3f ms\n", tiled_time);
    printf("\nSpeedup: %.2fx\n", naive_time / tiled_time);
    
    cudaFree(d_input);
    cudaFree(d_output);
    free(h_input);
    
    return 0;
}

In [ ]:
!nvcc -arch=sm_75 -O3 convolution_2d.cu -o convolution_2d && ./convolution_2d

### Week 2 Memory Summary

| Memory Type | Size | Best For | Access |
|------------|------|----------|--------|
| **Global** | GBs | Large data | All threads |
| **Shared** | 48-164KB/SM | Data reuse in block | Per-block |
| **Constant** | 64KB | Broadcast (all read same) | Read-only |
| **Texture** | Via cache | 2D spatial locality | Read-only |
| **Registers** | ~255/thread | Per-thread data | Per-thread |

**Optimization Techniques Learned:**
1. ✅ Memory coalescing (consecutive threads → consecutive addresses)
2. ✅ Structure of Arrays (SoA) over Array of Structures (AoS)
3. ✅ Shared memory for data reuse
4. ✅ `__syncthreads()` for thread synchronization
5. ✅ Padding to avoid bank conflicts
6. ✅ Constant memory for broadcast data

---

## Part 2: Python/Numba (Optional)

In [ ]:
# Setup
import subprocess, sys
try:
    import google.colab
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "numba"])
except ImportError:
    pass

from numba import cuda
import numpy as np

print(f"CUDA Device: {cuda.get_current_device().name.decode()}")

In [ ]:
# Note: Numba doesn't have direct constant memory support like CUDA C++
# But you can pass small read-only arrays as kernel arguments

@cuda.jit
def apply_filter(input_arr, output_arr, filter_arr):
    idx = cuda.grid(1)
    n = input_arr.size
    filter_size = filter_arr.size
    
    if idx < n:
        sum_val = 0.0
        for i in range(filter_size):
            input_idx = (idx + i) % n
            sum_val += input_arr[input_idx] * filter_arr[i]
        output_arr[idx] = sum_val

# Test
N = 1 << 20
FILTER_SIZE = 9

input_arr = cuda.to_device(np.ones(N, dtype=np.float32))
output_arr = cuda.device_array(N, dtype=np.float32)
filter_arr = cuda.to_device(np.ones(FILTER_SIZE, dtype=np.float32) / FILTER_SIZE)

threads = 256
blocks = (N + threads - 1) // threads

apply_filter[blocks, threads](input_arr, output_arr, filter_arr)
cuda.synchronize()

result = output_arr.copy_to_host()
print(f"Result[0] = {result[0]:.4f} (expected: 1.0)")
print("\nNote: In Numba, small read-only arrays are automatically")
print("optimized by the compiler, similar to constant memory.")